In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Hyper-parameters
num_epochs = 50
batch_size = 64
learning_rate = 0.001

# Convolutional neural network
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResBlock, self).__init__()
        self.conv = nn.Sequential(
                       nn.Conv2d(in_channels, out_channels, stride = stride, kernel_size = 3, padding = 1),
                       nn.BatchNorm2d(out_channels),
                       nn.ReLU())
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        # Batch normalisation used on convolution layers

    def forward(self, x):
        residual = x
        out = self.conv(x)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 102):
        super(ResNet, self).__init__()
        self.in_channels = 32
        self.conv = nn.Sequential(
                       nn.Conv2d(3, 32, kernel_size = 3, stride = 2, padding = 1),
                       nn.BatchNorm2d(32))
        self.layer0 = self.make_layer(block, 32, layers[0], stride = 1)
        self.layer1 = self.make_layer(block, 64, layers[1], stride = 2)
        self.layer2 = self.make_layer(block, 128, layers[2], stride = 2)
        self.layer3 = self.make_layer(block, 256, layers[3], stride = 2)
        self.layer4 = self.make_layer(block, 512, layers[4], stride = 2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.drop = nn.Dropout(0.2)
        
    def make_layer(self, block, out_channels, blocks, stride = 1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                            nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride),
                            nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        print(x.shape)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.drop(x)
        
        return x
    
# Create model and push to device
model = ResNet(ResBlock, [1,1,1,1,1]).to(device)

# Get loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define transforms with data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomVerticalFlip(p=0.05),
    transforms.RandomHorizontalFlip(p=0.1),
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Flower102: Unknown number and various sized color images in 102 classes, with 40 to 258 images per class
train_dataset = torchvision.datasets.Flowers102(root='./data', split='train',
                                                download=True, transform=train_transform)
test_dataset = torchvision.datasets.Flowers102(root='./data', split='test',
                                               download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False)

def train(model, train_loader, criterion, optimizer, device, epoch, smoothing=0.1):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Apply label smoothing
        num_classes = model.fc2.out_features
        one_hot_labels = torch.zeros(labels.size(0), num_classes).to(device)
        one_hot_labels.scatter_(1, labels.view(-1, 1), 1)
        one_hot_labels = one_hot_labels * (1 - smoothing) + smoothing / num_classes

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, one_hot_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    loss = train_loss / len(train_loader)

    return loss, acc

# Train the model
train_losses, train_accs, valid_losses, valid_accs = [], [], [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device, epoch)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    if ((epoch + 1) % 5 == 0):
      valid_loss, valid_acc = test(model, test_loader, criterion, device)
      valid_losses.append(valid_loss)   
      valid_accs.append(valid_acc)
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.2f}%')
    else:
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')

# Save the model
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)
     

torch.Size([64, 512, 3, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (98304x3 and 512x2048)